# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

#### COTAÇÃO DÓLAR

In [2]:
navegador = webdriver.Chrome()

navegador.get('https://www.google.com/') # Entrar no google
navegador.find_element(By.XPATH, #xpath é o caminho do elemento no site do google
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input'
     ).send_keys( 'cotação dólar hoje') #send_keys digita oq queremos 

navegador.find_element(By.XPATH, 
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input'
     ).send_keys(Keys.ENTER) # Apertar Enter

cotacao_dolar = navegador.find_element(By.XPATH, 
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]' #xpath do elemento cotação
     ).get_attribute('data-value') #Pegar a cotação indicada
print(cotacao_dolar)

5.0891


#### COTAÇÃO EURO



In [3]:
navegador.get('https://www.google.com/') # Entrar no google
navegador.find_element(By.XPATH, #xpath é o caminho do elemento no site do google
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input'
     ).send_keys( 'cotação euro hoje') #send_keys digita oq queremos 

navegador.find_element(By.XPATH, 
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input'
     ).send_keys(Keys.ENTER) # Apertar Enter

cotacao_euro = navegador.find_element(By.XPATH, 
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]' #xpath do elemento cotação
     ).get_attribute('data-value') #Pegar a cotação indicada
print(cotacao_euro)

5.5169999999999995


#### COTAÇÃO OURO

In [4]:
navegador.get('https://www.melhorcambio.com/ouro-hoje') # Entrar na cotação do ouro no site melhor cambio
cotacao_ouro = navegador.find_element(
    By.XPATH, '//*[@id="comercial"]').get_attribute('value') # Pegar a cotação
cotacao_ouro = cotacao_ouro.replace(',', '.') # Substituir a vírgula pelo ponto
print(cotacao_ouro)

310.99


In [5]:
navegador.quit()

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [6]:
tabela = pd.read_excel('Produtos.xlsx')
tabela.head(5)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.4,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.0,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.7,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.7,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.9,34200.000


- Atualizando as cotações na tabela

In [7]:
# Localizar na tabela, as cotações em que as linhas da colunao moeda == dolar, e substituir por cotação dolar
# O float é usado para garantir que o tipo seja numérico, caso tenha raspado uma string

tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar) 
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro) 
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro) 
tabela.head()

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.0891,4999.95,1.4,6999.930
1,Carro Renault,4500.00,Euro,5.5170,27000.00,2.0,54000.000
2,Notebook Dell,899.99,Dólar,5.0891,4499.95,1.7,7649.915
3,IPhone,799.00,Dólar,5.0891,3995.00,1.7,6791.500
4,Carro Fiat,3000.00,Euro,5.5170,18000.00,1.9,34200.000


- Refazendo os cálculos de Preço de compra e Preço de venda

In [8]:
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']
tabela.head()

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.0891,5089.049109,1.4,7124.668753
1,Carro Renault,4500.00,Euro,5.5170,24826.500000,2.0,49653.000000
2,Notebook Dell,899.99,Dólar,5.0891,4580.139109,1.7,7786.236485
3,IPhone,799.00,Dólar,5.0891,4066.190900,1.7,6912.524530
4,Carro Fiat,3000.00,Euro,5.5170,16551.000000,1.9,31446.900000


### Agora vamos exportar a nova base de preços atualizada

In [9]:
tabela.to_excel('Produtos_atualizado.xlsx', index = False)

## Instalaçao das bibliotecas 

In [10]:
!pip install selenium

In [11]:
!pip install dash==1.20.0 